All instructions are provided for R. I am going to reproduce them in Python as best as I can.

# Preface

From the textbook, p. 201:
> We will now consider the `Boston` housing data set, from the MASS library.

In [1]:
import numpy as np
import pandas as pd
import scipy.stats
from sklearn.utils import resample
from sklearn.datasets import load_boston

In [2]:
boston = load_boston()
# Because this exercise requires all features to be in one table,
# loading straight from the CSV is easier.
bostondf = pd.read_csv(boston.filename, skiprows=1)

predictors = (bostondf.drop('CRIM', axis='columns')
                      .assign(intercept=1)
)
target = bostondf.CRIM

# (a)

From the textbook, p. 201:
> Based on this data set, provide an estimate for the population mean of `medv`. Call this estimate $\hat{\mu}$.

In [3]:
mu = predictors.MEDV.mean()
mu

22.532806324110698

# (b)

From the textbook, p. 201:
> Provide an estimate of the standard error of $\hat{\mu}$. Interpret this result.<br>*Hint: We can compute the standard error of the sample mean by dividing the sample standard deviation by the square root of the number of observations.*

In [4]:
predictors.MEDV.std() / np.sqrt(len(predictors.MEDV))

0.4088611474975351

# (c)

From the textbook, p. 201:
> Now estimate the standard error of $\hat{\mu}$ using the bootstrap. How does this compare to your answer from (b)?

In [5]:
N_BOOTSTRAPS = 1000
means = []
for i in range(N_BOOTSTRAPS):
  bootstrap_sample = resample(predictors, replace=True)
  means.append(bootstrap_sample.MEDV.mean())
means = np.array(means)
se_estimate = means.std(ddof=1)

print(se_estimate)

0.38611410242820604


It is quite close to the theoretical value.

# (d)

From the textbook, p. 201:
> Based on your bootstrap estimate from (c), provide a 95% confidence interval for the mean of `medv`. Compare it to the results obtained using `t.test(Boston$medv)`.<br>*Hint: You can approximate a 95% confidence interval using the formula $ [\hat{\mu} - 2 \text{SE}(\mu), \; \hat{\mu} + 2 \text{SE}(\mu)] $.*

In [6]:
print(f'Confidence interval for the mean (bootstrap): {(mu - 2*se_estimate, mu + 2*se_estimate)}')

# I could not find anything that exactly matches R's t.test() output,
# so I reproduced the confidence interval calculation.
confidence = 0.95
se = scipy.stats.sem(predictors.MEDV)
h = se * scipy.stats.t.ppf((1 + confidence) / 2., len(predictors.MEDV))
print(f'Confidence interval for the mean (bootstrap): {(mu - h, mu + h)}')

Confidence interval for the mean (bootstrap): (21.760578119254287, 23.30503452896711)
Confidence interval for the mean (bootstrap): (21.729531828250774, 23.336080819970622)


They match up to the fourth digit.

# (e)

From the textbook, p. 201:
> Based on this data set, provide an estimate, $\hat{\mu}_{med}$, for the median value of `medv` in the population.

In [7]:
predictors.MEDV.median()

21.2

# (f)

From the textbook, p. 201:
> We now would like to estimate the standard error of $\hat{\mu}_{med}$. Unfortunately, there is no simple formula for computing the standard error of the median. Instead, estimate the standard error of the median using the bootstrap. Comment on your findings.

In [8]:
N_BOOTSTRAPS = 1000
medians = []
for i in range(N_BOOTSTRAPS):
  bootstrap_sample = resample(predictors, replace=True)
  medians.append(bootstrap_sample.MEDV.median())
medians = np.array(medians)
se_estimate = medians.std(ddof=1)

print(se_estimate)

0.38040908384405175


The standard error looks quite small relative to the estimate.

# (g)

From the textbook, p. 201:
> Based on this data set, provide an estimate for the tenth percentile of `medv` in Boston suburbs. Call this quantity $\hat{\mu}_{0.1}$. (You can use the `quantile()` function.)

In [9]:
predictors.MEDV.quantile(0.1)

12.75

# (h)

From the textbook, p. 201:
> Use the bootstrap to estimate the standard error of $\hat{\mu}_{0.1}$. Comment on your findings.

In [10]:
N_BOOTSTRAPS = 1000
q01s = []
for i in range(N_BOOTSTRAPS):
  bootstrap_sample = resample(predictors, replace=True)
  q01s.append(bootstrap_sample.MEDV.quantile(0.1))
q01s = np.array(q01s)
se_estimate = q01s.std(ddof=1)

print(se_estimate)

0.4980623013801145


The standard error is still relatively small. It makes 95% confidence interval of $12.75 \pm 2$, which might be good or bad, depending on the application.